<a href="https://colab.research.google.com/github/ortizfram/predicting-stock-prices/blob/main/prediciton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Description : 
#     Get predicitons from Amazon Stock Price, using Daily Clousure price in USD
#     using NeuralNetwork
#     the data is from the Yahoo Finance API (2001-01-02 / 2023-02-23)

In [8]:
# install import libraries

# !pip install pandas-datareader tensorflow scikit-learn
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
import pandas_datareader as web

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [3]:
# Load data

# data = https://finance.yahoo.com/quote/AMZN/history?p=AMZN  
#       get this data from Yahoo Finance API.

df = pd.read_csv('/content/AMZN.csv')

df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2001-01-02,0.790625,0.800000,0.681250,0.693750,0.693750,184070000
1,2001-01-03,0.681250,0.893750,0.678125,0.878125,0.878125,293608000
2,2001-01-04,0.850000,0.878125,0.771875,0.775000,0.775000,212410000
3,2001-01-05,0.775000,0.793750,0.678125,0.728125,0.728125,175976000
4,2001-01-08,0.721875,0.778125,0.687500,0.746875,0.746875,208894000
...,...,...,...,...,...,...,...
5566,2023-02-16,99.209999,100.629997,98.099998,98.150002,98.150002,56339200
5567,2023-02-17,97.800003,97.940002,95.650002,97.199997,97.199997,59967700
5568,2023-02-21,95.339996,95.610001,94.269997,94.580002,94.580002,56580400
5569,2023-02-22,95.099998,97.010002,94.800003,95.790001,95.790001,59534100


In [4]:
# Prepare Data 

# initialize scaler
#     scale all the values that  we have so that it fit's betweet (0 and 1)
scaler = MinMaxScaler(feature_range=(0,1))

# fit the Target column
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1,1))

# how many days i want to base my prediction on 
#     (look back) -> to define what the $ is gonna be for the next day
prediction_days = 60

X_train = []
y_train = []

# Start from 60 till the lenght of scaled_data.
for x in range(prediction_days, len(scaled_data)):
  # Append 60 values, next value is training sample
  #    so our model can learn to predict the 61st value.
  X_train.append(scaled_data[x-prediction_days:x, 0])
  #    61st value
  y_train.append(scaled_data[x,0])

# Convert to train sets to Numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)
# reshape X_train so that is working w/ the Neural Network
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [6]:
# Buil the Model

# Initialize
model = Sequential()

# specify the layers
#     always add: 1 LSTM layer, 1 Dropout layer.... Finally 1 Dense layer
#     this last layer is gonna be the : Stock Price Prediction.
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))#Prediction of next clsoing value

# compile the model
model.compile(optimizer='adam',loss='mean_squared_error')
# Fit the model
#   same date 25 times, number of samples 32
model.fit(X_train, y_train, epochs=25, batch_size=32)

173/173 [==============================] - 18s 102ms/step - loss: 8.5574e-04
Epoch 16/25
173/173 [==============================] - 22s 130ms/step - loss: 8.4979e-04
Epoch 17/25
173/173 [==============================] - 21s 122ms/step - loss: 8.8168e-04
Epoch 18/25
173/173 [==============================] - 18s 103ms/step - loss: 9.2124e-04
Epoch 19/25
173/173 [==============================] - 20s 113ms/step - loss: 8.2286e-04
Epoch 20/25
173/173 [==============================] - 18s 105ms/step - loss: 7.0643e-04
Epoch 21/25
173/173 [==============================] - 18s 105ms/step - loss: 7.8027e-04
Epoch 22/25
173/173 [==============================] - 19s 109ms/step - loss: 7.9996e-04
Epoch 23/25
173/173 [==============================] - 18s 104ms/step - loss: 7.6227e-04
Epoch 24/25
173/173 [==============================] - 19s 111ms/step - loss: 6.8725e-04
Epoch 25/25
173/173 [==============================] - 18s 103ms/step - loss: 6.9640e-04


In [ ]:
# Test Model Accuracy on Existing Data

# Load Test data
#     we have this data already. But the model haven't seen it yet 
test_start = dt.datetime(2001,1,2)
test_end = dt.datetime.now()

company = 'AMZN'
test_data = web.DataReader(company, 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((df['Close'], test_data['Close']), axis=0)
model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values